In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
%matplotlib inline
import math
from numpy import linalg as LA

from scipy.linalg import eig, eigh
from scipy.sparse.linalg import eigs, eigsh
np.set_printoptions(linewidth=np.inf)

import time


In [2]:
##implementar el criterio para esocger la k
def serchk(n,M): # n es el vector de estados,
                 # M es la longitud del vector o lo sitios de ocupacion
        
    k=0                    #iniciamos en k=0
    for i in range(M):   ##Recorreremos todas las posiciones M para checar las j>i 
        k=i              ###nuestro prospecto a ser k
        #print("1 for ",k)
        flag=0             ##definimos unna bandera que si es 
                                    #1=> significa que encontro un n_{j} tal que los de adelante son 0 
                                    #0=> no lo ha encontrado
        for j in range(k+1,M-1):  #Revisaremos los n_{j} j>i si son cero
            #print("2 for ", n[j])
            if n[j]!=0.0:        # si se encuentra una n_{j} != 0
                break                   #ya no revise lo demas
            elif j==M-2:         #si j llega al penultimo valor entonces 
                flag=1                  #ya lo encontro
            #print("bandera ",flag)
        if flag==1:              # si la bandera es 1, es decir ya encontro la k
            k=i                  # entonces esa i es la k
            break                # ya no hace falta revisar más
    if k==(M-1):           #si no encontro ningún valor significa que tomaremos el penultimo valor
        k=M-2             #entonces k=M-1
    return k 

In [3]:
def base_fock(N,M):
    #D=math.factorial(N+M-1)/(math.factorial(N)*math.factorial(M-1))
    D=math.comb(N+M-1,N) # Es la dimension del espacio de Hilbert
    DM=np.zeros((D,M)) #Aqui se colocarán las bases
    #N=3  #numero de particlas
    #M=3  #Numero de sitios
    n=np.zeros(M)  #Este representará los estaods
    n[0]=N           # Colocamos en el lugar K=0 el número de particulas |N,0,0,0,0,...,0>

    DM[0,:]=n        #Colocamos el primer estado en una matriz
    l=0              # Necesitamos un contador para colocar los demas estados
    while n[-1]!=N:
        l=l+1        # variamos el contador
        k=serchk(n,M) #buscamos la k

        n[k]=n[k]-1 # Primer paso del cambio n_{k}=n_{k}-1


        ## Compienza el segundo paso de la suma n_{k+1}=n_{k}-1
        n[k+1]=N
        for i in range(k+1): #se le coloca un +1 para que pueda acceder al menos a la posicion 0
            n[k+1]=n[k+1]-n[i]
        ### Cambiamos por ceros los n_{i} tales que i >= k+2
        for i in range(k+2,M):
            n[i]=0          
        
        DM[l,:]=n  #Agregamos el estado n al la fila l
    return DM

def base_2_ex(N):
    base_2_extendida=np.zeros(((N+1)**2,2))
    l=0
    for i in range(N+1):
        for j in range(N+1):
            base_2_extendida[l,0]=i
            base_2_extendida[l,1]=j
            l+=1
    return base_2_extendida

In [4]:
### Operadores de ascenso y descenso y numero

def ascenso_i(estado,i,N):
    part=estado[i]+1
    norm=np.sqrt(estado[i]+1)
    if part>N:
        part=0
        norm=0
    estado[i]=part
    return norm

def descenso_i(estado,i,N):
    part=estado[i]-1
    norm=np.sqrt(estado[i])
    if part<0:
        part=0
        norm=0
    estado[i]=part
    return  norm

def numero_i(estado,i):
    return estado[i]

In [5]:
def buscador(estado,N,M):
    i0=N #indice para ayudarnos a que este fijo el num de particulas disponibles
    i1=0 #indice para ayudarnos a restar
    Sitios=2 #Numero de sitios que se quitan +1 (al principio se quita un sitio por eso es 2)

    contador=0
    
    for i in range(M):
        while (i0-i1)!=estado[i]:
            contador=contador+math.comb(M-Sitios+i1,i1)
            i1+=1
        i0=i1
        i1=0
        Sitios+=1
    return contador
def buscador_ext_bas(vec_base_ext,N):
    return np.int64(vec_base_ext[0]*(N+1) + vec_base_ext[1])


In [6]:
#Hamiltoniano Cinético at_{m}*a_{m+1} + at_{m+1}*a_{m}
def H_J(J,N,M,D,base):
    H=np.zeros((D,D))
    #Esta es la parte     at_{m}*a_{m+1} + at_{m+1}*a_{m}
    for l in range(D):#Usaremos cada elemento de la base
        for mu in range(M):
            n=copy.deepcopy(base[l]) # Es el vector con el que se trabajara
            if mu==M-1:
                a=descenso_i(n,0,N)
                at=ascenso_i(n,mu,N)
            else:
                a=descenso_i(n,mu+1,N)
                at=ascenso_i(n,mu,N)
            if at*a != 0:
                col=buscador(n,N,M)
                H[col,l]+=-J*at*a
                H[l,col]+=-J*at*a
    return H

def H_U(U,N,M,D,base):
    H=np.zeros((D,D))
    for l in range(D):
        n=copy.deepcopy(base[l])
        for i in range(M):
            H[l,l]+=U*0.5*n[i]*(n[i]-1)
    return H


In [7]:
def evolucion_temporal(t, Vec_0_E,D):
    Vec_final=np.zeros(D,dtype=np.complex_)
    exponencial=np.exp(-1j*t*evals)
    for i in range(D):
        Vec_final[i]=exponencial[i]*Vec_0_E[i]
    return Vec_final

In [8]:
def rho_c_reducida_ex(n_nu,n_mu,N,D,D_ext,bases,base_2_extendida,rho_c):
    rho_c_reducida_exten=np.zeros((D_ext,D_ext),dtype=np.complex_)
    
    ### Revisamos cada uno de los elemtnos de la base extendida |n_{\nu}^{i},n_{\mu}^{i}><n_{\nu}^{j},n_{\mu}^{j}|
    for i in range(D_ext):
        for j in range(D_ext):
            #### Esta es la primera delta de dirac (conservacion de particulas en la traza) --->
            ####  ---> \delta_{ n_{\nu}^{i}+n_{\mu}^{i} , n_{\nu}^{j}+n_{\mu}^{j} }
            if np.sum(base_2_extendida[i])==np.sum(base_2_extendida[j]):
            ### Aqui es la segunda delta de dirac (Conservacion de particulas sin traza) --->
            ### ----->\delta_{n_{1}^{i}+n_{2}^{i}+...+n_{M}^{i}} , N}
                for k in range(D): #Tecnicamente ya sabemos quienes cumplen eso son los de la base de fock
                    # Reviamos que elementos de la base de fock tienen n_{\nu}^{i} y n_{\mu}^{i} simultaneamente de la base extendida para i
                    # el k-esimo elemento es <k|\rho(t)| NOSE >
                    if (bases[k][n_nu]==base_2_extendida[i,0]) & (bases[k][n_mu]==base_2_extendida[i,1]):
                        #falta construir el NOSE que se compone de |n_{}^{i}...n_{\nu}^{j}...n_{\mu}^{j}>
                        #Con el <k| coinciden los que no son n_{\nu}^{j} n_{\mu}^{j}, entonces
                        NOSE=copy.deepcopy(bases[k])
                        NOSE[n_nu],NOSE[n_mu]=base_2_extendida[j][0],base_2_extendida[j][1]
                        rho_c_reducida_exten[i,j]=rho_c[buscador(bases[k],N,M),buscador(NOSE,N,M)]
    ### Traza parcial
    
    rho_c_reducida_exten_trans_par=np.zeros((D_ext,D_ext),dtype=np.complex_)
    ## Vamos a recorer cada elemento de la matriz de densidad reducida para transponerla
    for i in range(D_ext):
        for j in range(D_ext):
            ### Trasnponemos |n_{\nu}^{i},n_{\mu}^{i}><n_{\nu}^{j},n_{\mu}^{j}| --->
            ###     ------>  |n_{\nu}^{i},n_{\mu}^{j}><n_{\nu}^{j},n_{\mu}^{i}|
            Transpo1=buscador_ext_bas([base_2_extendida[i][0], base_2_extendida[j][1]],N)
            Transpo2=buscador_ext_bas([base_2_extendida[j][0], base_2_extendida[i][1]],N)
            
            rho_c_reducida_exten_trans_par[Transpo1,Transpo2]=rho_c_reducida_exten[i,j]
    return rho_c_reducida_exten_trans_par

In [9]:
#buscador([2,0,0],N,M)


In [45]:
start = time.time()


N=2 #Numero de sitios
M=3 #Numeor de Bosones
D=math.comb(N+M-1,N)
D_ext=(N+1)**2
J=0.01
U=1.0

bases=base_fock(N,M)
base_2_extendida=base_2_ex(N)

#Sitios de entrelazamiento
n_nu=0
n_mu=1

#Condicion inicial 
Vector_0_F=np.zeros(D)
Vector_0_F[0]=1

#tiempo
dt=0.1
Time_0=0
Time_f=100
times=np.arange(Time_0, Time_f, dt)

H=H_J(J,N,M,D,bases)+H_U(U,N,M,D,bases)

evals, Change_E_F=eigh(H)  #MAtriz cambio de base de Energias a Fock
Change_F_E=np.transpose(Change_E_F) # MAtriz cambio de base de Fock a Energias
Vector_0_E=np.dot(Change_F_E,Vector_0_F) #vector inicial para la base de energias
#DATA_N=np.zeros(len(times))

f=open("Negativity_python.dat","w")

for (index, t) in enumerate(times):

    Edo_F_t=np.dot(Change_E_F, evolucion_temporal(t,Vector_0_E,D)) #Es la evolución temporal
    Edo_F_t_conj=Edo_F_t.conjugate()

    #if t==0:
    #    print(Edo_F_t)
    rho_c=np.zeros((D,D),dtype=np.complex_) #Hacemos la matriz de densidad
    for i1 in range(D):
        for i2 in range(D):
            rho_c[i1,i2]=Edo_F_t[i1]*Edo_F_t_conj[i2]

    Eva=LA.eigvalsh(rho_c_reducida_ex(n_nu,n_mu,N,D,D_ext,bases,base_2_extendida,rho_c))

    Negativity=0
    for i in Eva:
        if i<0:
            Negativity+=abs(i)

    #DATA_N[index]=Negativity
    f.write(str(Negativity)+" "+str(t)+"\n")
f.close()

end = time.time()
print(end - start)

0.9866101741790771


In [46]:
#plt.plot(times, DATA_N)

In [47]:
# f=open("Negativity_python.dat","w")
# for i in range(len(times)):
#     f.write(str(DATA_N[i])+" "+str(times[i])+"\n")
# f.close()

In [48]:
Vector_0_E

array([-0.01599131, -0.01166007,  0.        ,  0.81641332,  0.        , -0.57712877])

In [53]:
Change_E_F[0:,0]

array([-0.01599131, -0.57712877, -0.57712877, -0.01599131, -0.57712877, -0.01599131])

In [50]:
Change_E_F

array([[-1.59913078e-02, -1.16600735e-02,  0.00000000e+00,  8.16413320e-01,  0.00000000e+00, -5.77128765e-01],
       [-5.77128765e-01, -4.08206660e-01,  7.07034675e-01, -5.83003677e-03,  1.00979199e-02,  1.59913078e-02],
       [-5.77128765e-01, -4.08206660e-01, -7.07034675e-01, -5.83003677e-03, -1.00979199e-02,  1.59913078e-02],
       [-1.59913078e-02,  5.83003677e-03,  1.00979199e-02, -4.08206660e-01, -7.07034675e-01, -5.77128765e-01],
       [-5.77128765e-01,  8.16413320e-01, -1.11022302e-16,  1.16600735e-02, -6.76542156e-17,  1.59913078e-02],
       [-1.59913078e-02,  5.83003677e-03, -1.00979199e-02, -4.08206660e-01,  7.07034675e-01, -5.77128765e-01]])